In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
import requests
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')


Libraries imported.


## Download and Data Wrangling

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

## Transform the Data into pandas Dataframe

In [5]:
# define the dataframe columns

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [12]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## FourSquare Venues

In [34]:
# The code was removed by Watson Studio for sharing.

In [14]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
#https://developer.foursquare.com/docs/resources/categories
#Latin American Restaurant = 4bf58dd8d48988d1be941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_Latam = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1c1941735')
newyork_venues_Latam.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Guacamole,40.874511,-73.910708,Mexican Restaurant
1,Marble Hill,40.876551,-73.91066,Taqueria Sinaloense,40.874574,-73.910687,Mexican Restaurant
2,Marble Hill,40.876551,-73.91066,Mi Lindo San Miguelito,40.880023,-73.906488,Mexican Restaurant
3,Marble Hill,40.876551,-73.91066,Picante Picante Mexican Restaurant,40.878252,-73.902936,Mexican Restaurant
4,Marble Hill,40.876551,-73.91066,Estrellita Poblana V,40.879687,-73.906257,Mexican Restaurant


In [17]:
newyork_venues_Latam.shape

(1620, 7)

In [18]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [19]:
map_newyork_Latam = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_Latam, 'red', map_newyork_Latam)

map_newyork_Latam

In [20]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [21]:
manhattan_grouped = newyork_venues_Latam.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,48,48,48,48,48,48
Carnegie Hill,42,42,42,42,42,42
Central Harlem,27,27,27,27,27,27
Chelsea,48,48,48,48,48,48
Chinatown,49,49,49,49,49,49
Civic Center,48,48,48,48,48,48
Clinton,49,49,49,49,49,49
East Harlem,47,47,47,47,47,47
East Village,50,50,50,50,50,50


## Neighborhood Analysis

In [22]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_Latam[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_Latam['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Bar,Breakfast Spot,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Deli / Bodega,Empanada Restaurant,Fast Food Restaurant,Food Stand,Food Truck,Latin American Restaurant,Mexican Restaurant,Seafood Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [23]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Bar,Breakfast Spot,Burger Joint,Burrito Place,Caribbean Restaurant,Chinese Restaurant,Deli / Bodega,Empanada Restaurant,Fast Food Restaurant,Food Stand,Food Truck,Latin American Restaurant,Mexican Restaurant,Seafood Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Battery Park City,0.00,0.020833,0.000000,0.000000,0.104167,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.020833,0.000000,0.666667,0.000000,0.187500,0.000000,0.000000,0.000000
1,Carnegie Hill,0.00,0.000000,0.000000,0.023810,0.023810,0.00,0.023810,0.000000,0.00,0.023810,0.00,0.023810,0.000000,0.714286,0.000000,0.166667,0.000000,0.000000,0.000000
2,Central Harlem,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.037037,0.888889,0.000000,0.037037,0.037037,0.000000,0.000000
3,Chelsea,0.00,0.000000,0.020833,0.000000,0.041667,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.041667,0.020833,0.562500,0.020833,0.250000,0.041667,0.000000,0.000000
4,Chinatown,0.00,0.020408,0.000000,0.020408,0.061224,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.673469,0.000000,0.204082,0.000000,0.000000,0.020408
5,Civic Center,0.00,0.020833,0.000000,0.020833,0.104167,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.625000,0.000000,0.208333,0.000000,0.000000,0.020833
6,Clinton,0.00,0.000000,0.000000,0.000000,0.061224,0.00,0.000000,0.000000,0.00,0.020408,0.00,0.040816,0.020408,0.653061,0.000000,0.183673,0.020408,0.000000,0.000000
7,East Harlem,0.00,0.000000,0.000000,0.000000,0.021277,0.00,0.021277,0.000000,0.00,0.021277,0.00,0.042553,0.000000,0.723404,0.000000,0.170213,0.000000,0.000000,0.000000
8,East Village,0.00,0.020000,0.000000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.740000,0.000000,0.180000,0.020000,0.000000,0.000000
9,Financial District,0.00,0.020833,0.000000,0.000000,0.083333,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.020833,0.000000,0.687500,0.000000,0.187500,0.000000,0.000000,0.000000


In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,American Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant
1,Carnegie Hill,Mexican Restaurant,Taco Place,Fast Food Restaurant,Food Truck,Burger Joint,Burrito Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
2,Central Harlem,Mexican Restaurant,Tex-Mex Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Food Truck,Fast Food Restaurant
3,Chelsea,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Tex-Mex Restaurant,Breakfast Spot,Seafood Restaurant,Latin American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant
4,Chinatown,Mexican Restaurant,Taco Place,Burrito Place,Vietnamese Restaurant,Burger Joint,Bar,Food Truck,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant


## Cluster Neighborhoods

In [26]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 1, 2, 0, 0, 0, 3, 3, 0], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Mexican Restaurant,Taco Place,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck,Fast Food Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Mexican Restaurant,Taco Place,Burrito Place,Vietnamese Restaurant,Burger Joint,Bar,Food Truck,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Mexican Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Caribbean Restaurant,Food Truck,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Taco Place,Fast Food Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck,American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Mexican Restaurant,Food Truck,Tex-Mex Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Fast Food Restaurant


In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Mexican Restaurant,Taco Place,Burrito Place,Vietnamese Restaurant,Burger Joint,Bar,Food Truck,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
10,Lenox Hill,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,Food Truck,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant
13,Lincoln Square,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Deli / Bodega,American Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant
14,Clinton,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Fast Food Restaurant,Tex-Mex Restaurant,Latin American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant
16,Murray Hill,Mexican Restaurant,Taco Place,Burrito Place,Food Stand,Tex-Mex Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck
18,Greenwich Village,Mexican Restaurant,Taco Place,Burrito Place,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Latin American Restaurant,Food Truck,Food Stand,Seafood Restaurant
21,Tribeca,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Bar,Tex-Mex Restaurant,Seafood Restaurant
23,Soho,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant
28,Battery Park City,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,American Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant
29,Financial District,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,Bar,American Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant


## Cluster 2

In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Mexican Restaurant,Taco Place,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck,Fast Food Restaurant
4,Hamilton Heights,Mexican Restaurant,Food Truck,Tex-Mex Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Fast Food Restaurant
5,Manhattanville,Mexican Restaurant,Taco Place,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck,Fast Food Restaurant
6,Central Harlem,Mexican Restaurant,Tex-Mex Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Food Truck,Fast Food Restaurant
25,Manhattan Valley,Mexican Restaurant,Food Truck,Bar,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Taco Place,Seafood Restaurant,Latin American Restaurant
26,Morningside Heights,Mexican Restaurant,Taco Place,Bar,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck


## Cluster 3

In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,Fast Food Restaurant,Latin American Restaurant,Food Truck,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant
17,Chelsea,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Tex-Mex Restaurant,Breakfast Spot,Seafood Restaurant,Latin American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant
24,West Village,Mexican Restaurant,Taco Place,Food Truck,Tex-Mex Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Burrito Place,Seafood Restaurant,Latin American Restaurant,American Restaurant
34,Sutton Place,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,Food Truck,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant
35,Turtle Bay,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck
39,Hudson Yards,Mexican Restaurant,Taco Place,Food Truck,Burrito Place,Fast Food Restaurant,Tex-Mex Restaurant,Latin American Restaurant,Bar,Vegetarian / Vegan Restaurant,Seafood Restaurant


## Cluster 4

In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Mexican Restaurant,Taco Place,Latin American Restaurant,American Restaurant,Caribbean Restaurant,Food Truck,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
3,Inwood,Mexican Restaurant,Taco Place,Fast Food Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant,Food Truck,American Restaurant
7,East Harlem,Mexican Restaurant,Taco Place,Food Truck,Fast Food Restaurant,Burrito Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant
8,Upper East Side,Mexican Restaurant,Taco Place,Fast Food Restaurant,Food Truck,Burger Joint,Burrito Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant
9,Yorkville,Mexican Restaurant,Taco Place,Fast Food Restaurant,Burger Joint,Tex-Mex Restaurant,Food Truck,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant
11,Roosevelt Island,Mexican Restaurant,Taco Place,Tex-Mex Restaurant,Food Truck,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant,Fast Food Restaurant
12,Upper West Side,Mexican Restaurant,Taco Place,Burrito Place,Food Truck,American Restaurant,Food Stand,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant,Latin American Restaurant
19,East Village,Mexican Restaurant,Taco Place,Burrito Place,Tex-Mex Restaurant,Bar,American Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Seafood Restaurant,Latin American Restaurant
20,Lower East Side,Mexican Restaurant,Taco Place,Burrito Place,Empanada Restaurant,Bar,American Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
22,Little Italy,Mexican Restaurant,Taco Place,Burrito Place,Vietnamese Restaurant,Burger Joint,Bar,Food Truck,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Seafood Restaurant
